In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import numpy as np
import torchvision
from torchvision import datasets, models, transforms as t
from fastprogress.fastprogress import master_bar, progress_bar
import matplotlib.pyplot as plt
import time
import os
import copy
from pathlib import Path

In [2]:
path = Path('../data/normal-vs-camouflage-clothes/8k_normal_vs_camouflage_clothes_images')

## Splitting data into train and validation split

In [ ]:
#We are taking 20 percent of data
valid_pct = int(7950*0.2)

def copy_files(files,dst_path):
    for o in files:o.rename(dst_path/o.name)

for cloth_type in ['camouflage_clothes','normal_clothes']:
    files = list((path/cloth_type).iterdir())
    for folder_name in ['train','valid']:
        (path/folder_name/cloth_type).mkdir(parents=True,exist_ok=True)
        copy_files(files[valid_pct:] if folder_name == 'train' else files[:valid_pct],path/folder_name/cloth_type)


## Creating dataset

In [3]:
trn_path = path/'train'
valid_path = path/'valid'

bs = 64
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
tfms = [[t.RandomResizedCrop(224),t.RandomHorizontalFlip(),t.ToTensor(),t.Normalize(*imagenet_stats)],
 [t.Resize(256),t.CenterCrop(224),t.ToTensor(),t.Normalize(*imagenet_stats)]]

trn_ds,valid_ds = [datasets.ImageFolder(o,transform=t.Compose(tfms[i])) for i,o in enumerate([trn_path,valid_path])]
trn_dl = DataLoader(trn_ds,batch_size=bs,shuffle=True,num_workers=4)
valid_dl = DataLoader(valid_ds,batch_size=bs,num_workers=4)

x,y = next(iter(trn_dl))

## Creating Model

In [4]:
class Flatten(nn.Module):
    def forward(self,x):
        return torch.flatten(x,1)

class MyResNet(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = models.resnet34(pretrained=True)
        self.body = nn.Sequential(*list(resnet.children())[:-2])
        self.head = nn.Sequential(nn.AdaptiveAvgPool2d(1),Flatten(),nn.Linear(512,2))
    
    def forward(self,x):
        x = self.body(x)
        return self.head(x)



## Creating fit method

In [13]:
#Borrowed from fastai2 library

bn_types = (torch.nn.modules.batchnorm.BatchNorm1d,torch.nn.modules.batchnorm.BatchNorm2d,torch.nn.modules.batchnorm.BatchNorm3d)
 
def set_bn_eval(m:nn.Module)->None:
    "Set bn layers in eval mode for all recursive children of `m`."
    for l in m.children():
        if isinstance(l, bn_types) and not next(l.parameters()).requires_grad:
            l.eval()
        set_bn_eval(l)

In [14]:
def fit(epochs,model,train_dl,valid_dl,loss_fn,opt,device=None,bn_eval=False):
    if device is None:
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    mb = master_bar(range(epochs))
    mb.write(['epoch','train_loss','valid_loss','trn_acc','val_acc'],table=True)
    model.to(device)

    for i in mb:    
        trn_loss,val_loss = 0.0,0.0
        trn_acc,val_acc = 0,0
        trn_n,val_n = len(train_dl.dataset),len(valid_dl.dataset)
        model.train()
        if bn_eval:set_bn_eval(model)
        for xb,yb in progress_bar(train_dl,parent=mb):
            xb,yb = xb.to(device), yb.to(device)
            out = model(xb)
            opt.zero_grad()
            loss = loss_fn(out,yb)
            _,pred = torch.max(out.data, 1)
            trn_acc += (pred == yb).sum().item()
            trn_loss += loss.item()
            loss.backward()
            opt.step()
        trn_loss /= mb.child.total
        trn_acc /= trn_n

        model.eval()
        with torch.no_grad():
            for xb,yb in progress_bar(valid_dl,parent=mb):
                xb,yb = xb.to(device), yb.to(device)
                out = model(xb)
                loss = loss_fn(out,yb)
                val_loss += loss.item()
                _,pred = torch.max(out.data, 1)
                val_acc += (pred == yb).sum().item()
        val_loss /= mb.child.total
        val_acc /= val_n

        mb.write([i,f'{trn_loss:.6f}',f'{val_loss:.6f}',f'{trn_acc:.6f}',f'{val_acc:.6f}'],table=True)        
        
    

## Training

In [6]:
loss_fn = F.cross_entropy

In [9]:
def freeze(model,bn_freeze=True):
    for name,param in model.named_parameters():
        if bn_freeze:
            param.requires_grad = False
        elif name.find('bn') == -1:
            param.requires_grad = False
            
def unfreeze(model):
    for param in model.parameters():
        param.requires_grad = True

def get_model(lrs=[1e-3,1e-3],bn_freeze=True):
    model = MyResNet()
    freeze(model.body,bn_freeze=bn_freeze)
    opt = optim.Adam([{'params': model.body.parameters(), 'lr':lrs[0]},
                {'params': model.head.parameters(), 'lr': lrs[1]}])
    return model,opt

def update_lr(lr,opt):
    opt.param_groups[0]['lr'] = lr/100
    opt.param_groups[1]['lr'] = lr
    

### Freeze the complete resnet body

In [8]:
lr = 1e-3
model,opt = get_model(lrs=[lr,lr],bn_freeze=True)
fit(2,model,trn_dl,valid_dl,loss_fn,opt)

epoch,train_loss,valid_loss,trn_acc,val_acc
0,0.161580,0.060043,0.944960,0.980189
1,0.083135,0.045104,0.973345,0.985535


### Freeze the complete resnet body and place BN layers in eval mode.

In [15]:
lr = 1e-3
model,opt = get_model(lrs=[lr,lr],bn_freeze=True)
fit(2,model,trn_dl,valid_dl,loss_fn,opt,bn_eval=True)

epoch,train_loss,valid_loss,trn_acc,val_acc
0,0.145801,0.048760,0.950936,0.987107
1,0.064337,0.038624,0.979163,0.987736


### Freeze the complete resnet body and place BN layers in eval mode and train the body at a lesser learning rate for the second epoch.

In [17]:
lr = 1e-3
model,opt = get_model(lrs=[lr,lr],bn_freeze=True)
fit(1,model,trn_dl,valid_dl,loss_fn,opt,bn_eval=True)

epoch,train_loss,valid_loss,trn_acc,val_acc
0,0.115747,0.044909,0.964774,0.987107


In [18]:
update_lr(lr/2,opt)
unfreeze(model)
fit(1,model,trn_dl,valid_dl,loss_fn,opt)

epoch,train_loss,valid_loss,trn_acc,val_acc
0,0.074065,0.024733,0.973581,0.991509


### Freeze the resnet body except fot BN layers

In [28]:
model,opt = get_model(lrs=[1e-3,1e-3],bn_freeze=False)
fit(2,model,trn_dl,valid_dl,loss_fn,opt)

epoch,train_loss,valid_loss,trn_acc,val_acc
0,0.105815,0.032677,0.960843,0.988365
1,0.052449,0.022727,0.982151,0.990881


### Freeze the resnet body except fot BN layers and try smaller leraning rate for the resnet body

In [12]:
model,opt = get_model(lrs=[lr,lr],bn_freeze=False)
fit(1,model,trn_dl,valid_dl,loss_fn,opt)
update_lr(lr/2,opt)
unfreeze(model)
fit(1,model,trn_dl,valid_dl,loss_fn,opt)

epoch,train_loss,valid_loss,trn_acc,val_acc
0,0.145204,0.052064,0.951329,0.983648


epoch,train_loss,valid_loss,trn_acc,val_acc
0,0.064199,0.020451,0.977669,0.993711


### Try adjusting the model

In [20]:
class AdaptiveConcatPooling(nn.Module):
    def forward(self,x):
        avg_pool = F.adaptive_avg_pool2d(x,1)
        max_pool = F.adaptive_max_pool2d(x,1)
        return torch.cat([avg_pool,max_pool],dim=1)

In [21]:
class MyResNet(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = models.resnet34(pretrained=True)
        self.body = nn.Sequential(*list(resnet.children())[:-2])
        self.head = nn.Sequential(AdaptiveConcatPooling(),Flatten(),nn.Linear(512*2,2))
    
    def forward(self,x):
        x = self.body(x)
        return self.head(x)

In [23]:
lr = 1e-3
model,opt = get_model(lrs=[lr,lr],bn_freeze=False)
fit(1,model,trn_dl,valid_dl,loss_fn,opt)
update_lr(lr/2,opt)
unfreeze(model)
fit(1,model,trn_dl,valid_dl,loss_fn,opt)

epoch,train_loss,valid_loss,trn_acc,val_acc
0,0.124573,0.033491,0.952115,0.987421


epoch,train_loss,valid_loss,trn_acc,val_acc
0,0.057533,0.032403,0.980893,0.990252


### Increasing the complexity of the model

In [24]:
class MyResNet(nn.Module):
    def __init__(self):
        super().__init__()
        nf = 512*2
        resnet = models.resnet34(pretrained=True)
        self.body = nn.Sequential(*list(resnet.children())[:-2])
        self.head = nn.Sequential(AdaptiveConcatPooling(),Flatten(),nn.BatchNorm1d(nf),nn.Dropout(p=0.25),
                      nn.Linear(nf,nf//2,bias=False),nn.ReLU(inplace=True),nn.BatchNorm1d(nf//2),nn.Dropout(p=0.75),
                      nn.Linear(nf//2,2,bias=False))
    
    def forward(self,x):
        x = self.body(x)
        return self.head(x)

In [25]:
lr = 1e-3
model,opt = get_model(lrs=[lr,lr],bn_freeze=False)
fit(1,model,trn_dl,valid_dl,loss_fn,opt)
update_lr(lr/2,opt)
unfreeze(model)
fit(1,model,trn_dl,valid_dl,loss_fn,opt)

epoch,train_loss,valid_loss,trn_acc,val_acc
0,0.097272,0.027744,0.966032,0.989308


epoch,train_loss,valid_loss,trn_acc,val_acc
0,0.056163,0.023533,0.979557,0.992767
